In [ ]:
#---lips detection with night vision based on current time
import cv2
import dlib
import numpy as np
import pygame
import os
import datetime

# Load pre-trained face and landmark detector models from dlib
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Function to calculate lip distance (vertical distance between top and bottom lip landmarks)
def lip_distance(shape):
    top_lip = shape[50:53]
    top_lip = np.concatenate((top_lip, shape[61:64]))

    bottom_lip = shape[56:59]
    bottom_lip = np.concatenate((bottom_lip, shape[65:68]))

    top_mean = np.mean(top_lip, axis=0)
    bottom_mean = np.mean(bottom_lip, axis=0)

    distance = abs(top_mean[1] - bottom_mean[1])
    return distance

# Initialize video capture
cap = cv2.VideoCapture(0)

# Variables for drowsiness detection
lip_distance_threshold = 18
frame_counter = 0
drowsy = False

# Initialize Pygame mixer for audio playback
pygame.mixer.init()
pygame.mixer.music.load('music.wav')  # Load the music file

# Function to apply night vision effect
def apply_night_vision(frame):
    # Split the frame into its color channels
    b, g, r = cv2.split(frame)

    # Enhance the green channel
    g = cv2.equalizeHist(g)

    # Merge back the channels
    night_vision_frame = cv2.merge((b, g, r))

    return night_vision_frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Check current time
    current_time = datetime.datetime.now().time()
    if current_time < datetime.time(6, 0) or current_time > datetime.time(19, 0):
        # Apply night vision effect to the frame
        frame = apply_night_vision(frame)
    else:
        # Use normal camera settings
        pass

    # Convert frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = detector(gray)

    for face in faces:
        # Predict facial landmarks
        landmarks = predictor(gray, face)

        # Convert landmarks to numpy array
        shape = np.zeros((68, 2), dtype=int)
        for i in range(0, 68):
            shape[i] = (landmarks.part(i).x, landmarks.part(i).y)

        # Calculate lip distance
        lip_dist = lip_distance(shape)

        # Draw landmarks on face
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)

        # Check for drowsiness
        if lip_dist > lip_distance_threshold:
            frame_counter += 1
            if frame_counter > 10:  # If lip movement is sustained for some frames
                if not drowsy:
                    pygame.mixer.music.play(-1)  # Play music in a loop
                    drowsy = True
        else:
            frame_counter = 0
            if drowsy:
                pygame.mixer.music.stop()  # Stop playing music
                drowsy = False

    # Display the frame
    cv2.imshow("Frame", frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and destroy any OpenCV windows
cap.release()
cv2.destroyAllWindows()
